In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct")

In [2]:
from langchain_community.vectorstores import Chroma

In [3]:
persist_directory = "../database/anwesha_chroma"
vectorstore = Chroma(
    persist_directory=persist_directory, embedding_function=embeddings)
retriever = vectorstore.as_retriever()

C:\Users\mdsaz\AppData\Local\Temp\ipykernel_15292\586100071.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)

In [5]:
from langchain import hub

In [6]:
prompt = hub.pull("rlm/rag-prompt")

In [7]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
rag_chain.invoke("অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?")

'অনুপম তার মামার চেয়ে **বছর ছয়েক** ছোট ছিল।'

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives
    | ChatGroq(model="moonshotai/kimi-k2-instruct",)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [11]:
from langchain.load import dumps, loads


def get_unique_union(documents: list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]


question = "অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
len(docs)

5

In [12]:
docs

[Document(metadata={'id': '65980bbb-7895-41b1-a526-012fe8172e86'}, page_content='৪৫। মাটু-বালকে কল্যাণী কার প্রতি ঈর্ষিত করেছে?\n\n(ক) মায়ের প্রতি     (খ) মাধুরীর প্রতি     (গ) ধনীর প্রতি     (ঘ) অনুপমের প্রতি\n\n৪৬। বিনুর অনুপমের পুত্রের বয়স কত ছিল?\n\n(ক) ২০ বছর     (খ) ২২ বছর     (গ) ২৫ বছর     (ঘ) ২৭ বছর\n\n৪৭। অনুপমের মেজো পুত্রের নাম কী?'),
 Document(metadata={'id': '65980bbb-7895-41b1-a526-012fe8172e86'}, page_content='১৪। মামা কেন মেয়ের পক্ষে পক্ষে করতেন?  \n(ক) ধনী (খ) গরিব (গ) প্রমীণ (ঘ) শখলে  \n\n১৫। অনুপমের বন্ধুর নাম কি?  \n(ক) সজল (খ) জ্যোতিষ (গ) হরিশ (ঘ) মণীষ  \n\n১৬। মেয়ের চেয়ে মামার বয়সে খুরবচাই ফার কাছ গুরত্ব?  \n(ক) হরিশের (খ) অনুপমের (গ) মামার (ঘ) ঘটকের'),
 Document(metadata={'id': '65980bbb-7895-41b1-a526-012fe8172e86'}, page_content="১০। 'অপরিচিতা' গল্পে মামার সাথে অনুপমের বয়সের পার্থক্য কত?  \n(ক) বছর চারেক (খ) বছর ছয়েক (গ) বছর আটেক (ঘ) বছর দশেক  \n\n১১। বন্যার সিপাহশালার কোনটি শীর্ষক করবেন?  \n(ক) অনুপম চরিত্র (খ) অনুপম সংলাপ  \n(গ) অনুপম রূপসালাপ (ঘ) অন

In [13]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'অনুপম তার মামার চেয়ে **ছয় বছরের ছোট** ছিল।'

In [17]:
sample_queries = [
    "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
    "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?",
    "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
]

expected_responses = [
    "শম্ভুনাথ",
    "মামাকে",
    "১৫ বছর"
]

In [18]:
from ragas import EvaluationDataset


dataset = []

for query, reference in zip(sample_queries, expected_responses):
    relevant_docs = retriever.invoke(query)
    response = final_rag_chain.invoke({"question": query})
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [rdoc.page_content for rdoc in relevant_docs],
            "response": response,
            "reference": reference,
        }
    )

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [19]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)

result

Evaluating:   0%|          | 0/9 [00:00<?, ?it/s]

{'context_recall': 0.6667, 'faithfulness': 0.3333, 'factual_correctness(mode=f1)': 0.3333}